# Download images with `sentinelsat`


The *Copernicus Open Access Hub* provides complete, free and open access to Sentinel-1, Sentinel-2, Sentinel-3 and Sentinel-5P user products.

`sentinelsat` makes searching, downloading and retrieving the metadata of Sentinel satellite images from the Copernicus Open Access Hub easy.



In [28]:
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from collections import OrderedDict
from pathlib import Path
from datetime import date
import geopandas as gpd
from IPython.display import display

print(f'sentinelsat : {sentinelsat.__version__}')

sentinelsat : 1.0.1


## Set directory

In [2]:
computer_path = '/Volumes/nbdid-sst-lbrat2104/'
grp_letter    = 'X'

# Directory for all work files
work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'

roi_path = f'{work_path}ROI/'

Path(roi_path).mkdir(parents=True, exist_ok=True)

## Set Region of Interest

In [7]:
roi_file      = f'{roi_path}extent_roi_32631.shp'
roi_file_json = f'{roi_path}extent_roi_4326.geojson'

print(f'ROI file : {roi_file}')

# Convert to WGS 84 if needed

roi_gdf = gpd.read_file(roi_file)

if roi_gdf.crs != 'epsg:4326':
    print(f'Old CRS : {roi_gdf.crs}')
    
    roi_gdf = roi_gdf.to_crs('epsg:4326')

    print((f'New CRS : {roi_gdf.crs}'))


# Write ROI file in GeoJSON format

roi_gdf.to_file(roi_file_json,  driver='GeoJSON')


ROI file : /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/ROI/extent_roi_32631.shp
Old CRS : epsg:32631
New CRS : epsg:4326


## Authentification

The Copernicus Open Access Hub and probably most Data Hubs require authentication.

You can provide your credentials with **SentinelAPI(your username , your password)**


In [49]:
user     = 'ndeffense'
password = 'lbrat2104'

# Connect to the API

api = SentinelAPI(user, password)

## Sorting & Filtering

In addition to the *search query keywords* sentinelsat allows filtering and sorting of search results before download. To simplify these operations sentinelsat offers the convenience functions `to_geojson()`, `to_dataframe()` and `to_geodataframe()` which return the search results as a GeoJSON object, Pandas DataFrame or a GeoPandas GeoDataFrame, respectively. `to_dataframe()` and `to_geodataframe()` require pandas and geopandas to be installed, respectively.

In this example we query Sentinel-2 scenes over a location and convert the query results to a Pandas DataFrame. The DataFrame is then sorted by cloud cover and ingestion date. We limit the query to first 5 results within our timespan and download them, starting with the least cloudy scene. Filtering can be done with all data types, as long as you pass the id to the download function.

In [50]:
start_date    = date(2021, 7, 1)
end_date      = date(2021, 7, 14)

dates = (start_date, end_date)

platform_name = 'Sentinel-2'

if platform_name == 'Sentinel-2':
    s2_level     = 'Level-2A'
    product_type = 'S2MSI2A'
    cloud_cover = (0,90)

elif platform_name == 'Sentinel-1':
    polarisation_mode =  'VV VH'

In [55]:
# Search by polygon, time, and SciHub query keywords

footprint = geojson_to_wkt(read_geojson(roi_file_json))

if platform_name == 'Sentinel-2':

    products = api.query(footprint,
                        date = dates,
                        platformname = platform_name,
                        processinglevel = s2_level,
                        producttype = product_type,
                        cloudcoverpercentage = cloud_cover)


# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

# sort and limit to first 5 sorted products

products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
#products_df_sorted = products_df_sorted[['cloudcoverpercentage','ingestiondate']]

display(products_df)

products_df = products_df[products_df['title'].str.contains('31UFS')]

#display(products_df_sorted.columns)


display(products_df)


#products_df_sorted = products_df_sorted[0:1][0:1]

products_df_sorted = products_df_sorted[0:1]['filename']

print(products_df_sorted)

# download sorted and reduced products

#api.download_all(products_df_sorted.index)

#api.download_all(['96b88d4d-216d-4451-a7f5-038296e58034	'])



,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid
dd59d338-c10d-4f7e-ab2d-93b32b5bb274,S2A_MSIL2A_20210711T104031_N0301_R008_T31UFR_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-11T10:40:31.024Z, Instrument: MS...",false,2021-07-11 13:49:56,2021-07-11 10:40:31.024,2021-07-11 10:40:31.024,2021-07-11 16:48:35.876,...,GS2A_20210711T104031_031608_N03.01,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS2_20210711T134956_S2021...,S2A_OPER_MSI_L2A_TL_VGS2_20210711T134956_A0316...,S2A_MSIL2A_20210711T104031_N0301_R008_T31UFR_2...,dd59d338-c10d-4f7e-ab2d-93b32b5bb274
c8484458-dcc2-4df7-b86c-433e57427bd4,S2A_MSIL2A_20210711T104031_N0301_R008_T31UFS_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-11T10:40:31.024Z, Instrument: MS...",false,2021-07-11 13:49:56,2021-07-11 10:40:31.024,2021-07-11 10:40:31.024,2021-07-11 16:47:37.009,...,GS2A_20210711T104031_031608_N03.01,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS2_20210711T134956_S2021...,S2A_OPER_MSI_L2A_TL_VGS2_20210711T134956_A0316...,S2A_MSIL2A_20210711T104031_N0301_R008_T31UFS_2...,c8484458-dcc2-4df7-b86c-433e57427bd4
3c7677f8-53c5-42a3-b9b6-3d4f18522a4c,S2B_MSIL2A_20210709T104619_N0301_R051_T31UFS_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-09T10:46:19.024Z, Instrument: MS...",false,2021-07-09 14:23:18,2021-07-09 10:46:19.024,2021-07-09 10:46:19.024,2021-07-09 17:59:20.908,...,GS2B_20210709T104619_022671_N03.01,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20210709T142318_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20210709T142318_A0226...,S2B_MSIL2A_20210709T104619_N0301_R051_T31UFS_2...,3c7677f8-53c5-42a3-b9b6-3d4f18522a4c
56ff792c-8b3d-472d-a924-2aca10d4fcfb,S2B_MSIL2A_20210709T104619_N0301_R051_T31UFR_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-09T10:46:19.024Z, Instrument: MS...",false,2021-07-09 14:23:18,2021-07-09 10:46:19.024,2021-07-09 10:46:19.024,2021-07-09 17:43:59.361,...,GS2B_20210709T104619_022671_N03.01,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20210709T142318_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20210709T142318_A0226...,S2B_MSIL2A_20210709T104619_N0301_R051_T31UFR_2...,56ff792c-8b3d-472d-a924-2aca10d4fcfb
cd6e3f34-2582-426a-95e4-a157b1437484,S2B_MSIL2A_20210706T103629_N0301_R008_T31UFR_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-06T10:36:29.024Z, Instrument: MS...",false,2021-07-06 12:58:29,2021-07-06 10:36:29.024,2021-07-06 10:36:29.024,2021-07-06 18:25:36.017,...,GS2B_20210706T103629_022628_N03.01,S2MSI2A,2017-013A,ASCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20210706T125829_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20210706T125829_A0226...,S2B_MSIL2A_20210706T103629_N0301_R008_T31UFR_2...,cd6e3f34-2582-426a-95e4-a157b1437484
d6a16dff-93e5-4351-b079-51cf29ef46b7,S2B_MSIL2A_20210706T103629_N0301_R008_T31UFS_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-06T10:36:29.024Z, Instrument: MS...",false,2021-07-06 12:58:29,2021-07-06 10:36:29.024,2021-07-06 10:36:29.024,2021-07-06 18:10:42.760,...,GS2B_20210706T103629_02262

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid
c8484458-dcc2-4df7-b86c-433e57427bd4,S2A_MSIL2A_20210711T104031_N0301_R008_T31UFS_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-11T10:40:31.024Z, Instrument: MS...",false,2021-07-11 13:49:56,2021-07-11 10:40:31.024,2021-07-11 10:40:31.024,2021-07-11 16:47:37.009,...,GS2A_20210711T104031_031608_N03.01,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS2_20210711T134956_S2021...,S2A_OPER_MSI_L2A_TL_VGS2_20210711T134956_A0316...,S2A_MSIL2A_20210711T104031_N0301_R008_T31UFS_2...,c8484458-dcc2-4df7-b86c-433e57427bd4
3c7677f8-53c5-42a3-b9b6-3d4f18522a4c,S2B_MSIL2A_20210709T104619_N0301_R051_T31UFS_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-09T10:46:19.024Z, Instrument: MS...",false,2021-07-09 14:23:18,2021-07-09 10:46:19.024,2021-07-09 10:46:19.024,2021-07-09 17:59:20.908,...,GS2B_20210709T104619_022671_N03.01,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20210709T142318_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20210709T142318_A0226...,S2B_MSIL2A_20210709T104619_N0301_R051_T31UFS_2...,3c7677f8-53c5-42a3-b9b6-3d4f18522a4c
d6a16dff-93e5-4351-b079-51cf29ef46b7,S2B_MSIL2A_20210706T103629_N0301_R008_T31UFS_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2021-07-06T10:36:29.024Z, Instrument: MS...",false,2021-07-06 12:58:29,2021-07-06 10:36:29.024,2021-07-06 10:36:29.024,2021-07-06 18:10:42.760,...,GS2B_20210706T103629_022628_N03.01,S2MSI2A,2017-013A,ASCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20210706T125829_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20210706T125829_A0226...,S2B_MSIL2A_20210706T103629_N0301_R008_T31UFS_2...,d6a16dff-93e5-4351-b079-51cf29ef46b7


3c7677f8-53c5-42a3-b9b6-3d4f18522a4c    S2B_MSIL2A_20210709T104619_N0301_R051_T31UFS_2...
Name: filename, dtype: object


In [ ]:


footprint = geojson_to_wkt(read_geojson('search_polygon.geojson'))

products = api.query(footprint,
                     producttype='SLC',
                     orbitdirection='ASCENDING',
                     limit=10)

api.download_all(products)

In [33]:
tiles = ['33VUC', '33UUB']

query_kwargs = {
        'platformname': platform_name,
        'producttype': product_type,
        'date': dates}

products = OrderedDict()
for tile in tiles:
    kw = query_kwargs.copy()
    kw['tileid'] = tile
    pp = api.query(**kw)
    products.update(pp)

print(products)

OrderedDict()
